# SEQ2SEQ Modeling with NN.Transformer & Torchtext

In [1]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import dataset

In [18]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken:int, d_model:int, nhead:int, d_hid:int, nlayers:int, dropout:float = .5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        self.init_weights()
        
    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, +initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, +initrange)
    
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        '''
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        '''
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output
    
def generate_square_subsequent_mask(sz: int) -> Tensor:
    '''Generates an upper-triangular matrix of -inf, with zeros on diag'''
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model:int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(1e4 / d_model)))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe) # Don't calcuate gradients

    def forward(self, x: Tensor) -> Tensor:
        '''
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        '''
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


In [13]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    '''Converts raw text into a flat Tensor'''
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = 'cpu'

def batchify(data: Tensor, bsz: int) -> Tensor:
    '''Divides the data into bsz separate sequences'''
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz] # Remove extra sequences
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size) # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)
    

/Users/teddy/miniforge3/envs/Data/lib/python3.10/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/teddy/miniforge3/envs/Data/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


In [14]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data ,target

In [17]:
# Initialize hyperparameters
ntokens = len(vocab)
emsize = 200
d_hid = 200
nlayers = 2
nhead = 2
dropout = .2
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [20]:
# Learn
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


def train(model: nn.Module) -> None:
    model.train()
    total_loss = 0.0
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    
    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt: # last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0 :
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()
    return None

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval() # Start evaluate mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
        
        return total_loss / (len(eval_data) - 1)
    
# Save optimal model
best_val_loss = float('inf')
epochs = 3
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_mnodel = copy.deepcopy(model)
    scheduler.step()

/var/folders/qn/1t13bg7n0h1c7sv6dh677kq00000gn/T/ipykernel_35448/2719999763.py:29: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)


| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 224.00 | loss  8.39 | ppl  4403.28
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 232.22 | loss  7.04 | ppl  1146.99
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 256.81 | loss  6.56 | ppl   706.32
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 248.80 | loss  6.38 | ppl   589.19
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 252.59 | loss  6.24 | ppl   513.65
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 288.05 | loss  6.19 | ppl   488.97
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 252.53 | loss  6.13 | ppl   459.14
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 260.84 | loss  6.12 | ppl   454.19
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 261.96 | loss  6.04 | ppl   418.92
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 272.28 | loss  6.03 | ppl   413.87
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 283.84 | loss  5.90 | ppl   365.42
| epoch   

NameError: name 'tagets' is not defined

https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-8/03-perpexity

In [ ]:
# Evaluate Model
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)